# **Counting the number of occurrences of each character in the text.**
*The text is either loaded from
file or generated (student's choice). Each character is encoded with one
byte. The program should work correctly at least up to the size of the input data 4 million characters. The resulting array should be displayed on the screen.*

*For this lab it is necessary to compare with the CPU implementation in terms of execution speed, as well as to compare the results obtained element by element to confirm the correctness of the GPU-accelerated implementation. For a video card, the total time of copying input data into video memory, executing the kernel function and copying back the resulting data is measured, but the time of memory allocation and release is not measured.*

# **Подсчёт количества вхождений каждого символа в тексте.**
*Текст подгружается из
файла либо генерируется (на выбор студента). Каждый символ кодируется одним
байтом. Программа должна корректно работать хотя бы до размера входных
данных 4 млн. символов. Результирующий массив должен выводиться на экран.*

*Для этой лабораторной нужно сравнить с ЦП-реализацией по скорости выполнения, а также сравнить поэлементно полученные результаты, чтобы подтвердить корректность GPU-ускоренной реализации. Для видеокарты замеряется суммарное время копирования входных данных в видеопамять, выполнения функции-ядра и обратного копирования результирующих данных, но не замеряется время выделения и освобождения памяти.*



In [14]:
!pip install numba

In [15]:
import numpy as np
import time
from numba import cuda, jit, njit
import math

In [16]:
symbols = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
symbols_coded = {'a' : 0, 'b' : 1, 'c' : 2, 'd' : 3, 'e' : 4, 'f' : 5, 'g' : 6, 'h' : 7, 'i' : 8,
                'j' : 9, 'k' : 10, 'l' : 11, 'm' : 12, 'n' : 13, 'o' : 14, 'p' : 15, 'q' : 16,
                'r' : 17, 's' : 18, 't' : 19, 'u' : 20, 'v' : 21, 'w' : 22, 'x' : 23, 'y' : 24, 'z' : 25}
N = 4000000
text = np.random.choice(symbols, size = N)
print(text)
text_encoded = np.zeros(N, dtype = np.int8)
counter = np.zeros(26, dtype = np.int32)
for i in range(0, N):
  n = symbols_coded[text[i]]
  text_encoded[i] = n

['q' 'r' 'k' ... 's' 't' 't']


In [17]:
# core
@cuda.jit
def unique_elements_gpu(txt, cntr):
  i = cuda.grid(1)
  if i < len(txt):
    n = txt[i]
    cuda.atomic.add(cntr, n, int(1))

In [18]:
# CPU
#@njit()
def unique_elements_cpu(txt):
  counter = np.zeros(26, dtype = np.int32)
  for i in range(0, len(txt)):
    n = symbols_coded[txt[i]]
    counter[n] += 1
  return counter
print(text)


['q' 'r' 'k' ... 's' 't' 't']


In [31]:
gpu_res = np.zeros(26, dtype=np.int32)
text_encoded_device = cuda.to_device(text_encoded)
gpu_result_device = cuda.to_device(gpu_res)

In [32]:
block_size = 256
grid_size = math.ceil((N + block_size - 1) / (block_size))
t1 = time.time()
unique_elements_gpu[grid_size, block_size](text_encoded_device, gpu_result_device)
gpu_result = gpu_result_device.copy_to_host()
t2 = time.time()
print("Время выполнения на GPU: " + str(t2 - t1) + " с")
t1 = time.time()
cpu_result = unique_elements_cpu(text)
t2 = time.time()
print("Время выполнения на CPU: " + str(t2 - t1) + " с")
np.array_equal(gpu_result, cpu_result)

Время выполнения на GPU: 0.002438068389892578 с
Время выполнения на CPU: 9.782949447631836 с


True

In [33]:
print(gpu_result)
print(cpu_result)

[153813 153750 154905 154050 153722 153332 154236 154046 153955 153773
 153941 153394 154160 153514 153546 154334 153332 154348 153962 153279
 153467 153222 153828 153998 154234 153859]
[153813 153750 154905 154050 153722 153332 154236 154046 153955 153773
 153941 153394 154160 153514 153546 154334 153332 154348 153962 153279
 153467 153222 153828 153998 154234 153859]
